# Part 1: Crawler Module
Design and implement a crawler module that can collect paper title, author list, publication time, and abstract from PUBMED for a given keyword (i.e., HIV) within a pre-specified time window (that is, 01/01/2020 – 09/01/2020), and the retrieved data should be saved in the CSV format. 

In [ ]:
import pandas as pd
import numpy as np

Install biopython first by opening the anaconda prompt and typing in "pip install biopython".

In [ ]:
from Bio import Entrez
from urllib.error import HTTPError

In [ ]:
###Main function #################33
def Manue_search():
    inputKw = input("Please type in a keyword you are interested in: eg. HIV: ")
    inputTw = input("Please type in a time window for searching in format yyyy/mm/dd - yyyy/mm/dd: eg. 2020/08/29 - 2020/08/30: ")
    inputemail = input("Please type your email address:")    
    try:
        inputTw_min = inputTw.split("-")[0].strip()
        inputTw_max = inputTw.split("-")[1].strip()
        Papers_name = Extract_Raw_Info(inputKw, inputTw_min, inputTw_max, inputemail)
        Get_info_all(Papers_name)
    except HTTPError:
        print("Please retry, connection with NCBI is not stable")
    except:
        print("Make sure you type in the right format for time window, please rerun the current function.")


############### Get required information of one paper##############################33
#Input: all information for a paper
#Output: a list of extract infromation: title, author, publish date, and abstract
def Get_info_each(ppcon):
    #Organize the original file for one pubmed id
    fsm1 = list(map(lambda a: a.split("- "), ppcon))
    #Get the length of each element of data
    Each_len = list(map(len, fsm1))
    
    #Get the position of first line of each required information in extrated data list
    #Information: Ti: paper title, FAU: author list, DP: publication time, and AB: abstract
    ##extract first element--name of that line, in data
    Names = list(map(lambda a:a.strip(),next(zip(*fsm1))))
    DuidFun = lambda a: (i for i,value in enumerate(Names) if value == a)
    Ti_id = list(DuidFun("TI"))
    FAU_id = list(DuidFun("FAU"))
    DP_id = list(DuidFun("DP"))
    AB_id = list(DuidFun("AB"))

    #Get the information of FAU and DEP
    if len(FAU_id)!=0:
        FAU_final = "; ".join(list(fsm1[x][1] for x in FAU_id))
    else:
        FAU_final = None

    if len(DP_id)!=0:
        DP_final = fsm1[DP_id[0]][1]
    else:
        DP_final = None

    #Get the information of Ti and AB
    if len(Ti_id)!=0:
        Ti_end = Ti_id[0] + Each_len[Ti_id[0]+1:].index(2)
        Ti_full = [fsm1[Ti_id[0]][1]] + list(map(lambda a: a[0].strip(),fsm1[Ti_id[0] + 1:Ti_end + 1]))
        Ti_final = " ".join(Ti_full)
    else:
        Ti_final = None

    if len(AB_id)!=0:
        AB_end = AB_id[0] + Each_len[AB_id[0]+1:].index(2)
        AB_full = [fsm1[AB_id[0]][1]] + list(map(lambda a: a[0].strip(),fsm1[AB_id[0] + 1:AB_end + 1]))
        AB_final = " ".join(AB_full)
    else:
        AB_final = None
     
    #[Title = Ti_final, Author = FAU_final, Publish_date = DEP_final, Abstract = AB_final]
    return([Ti_final, FAU_final, DP_final, AB_final])


###########Interect with NCBI and get all infromation####################
#Note: this step might has have error with "Error 500: Internal Server Error", Ignor it, just retry
#During working hours, the line might be very crowded, you may get more 500 error T.T. Be patient, you can do it!

def Extract_Raw_Info(inputKw, inputTw_min, inputTw_max, inputemail):
    #############Search papers if with input query#######################################################
    #Tell NCBI who you are
    Entrez.email = inputemail
    handle_srch = Entrez.esearch(db = "pubmed", term = inputKw, usehistory = "y", datatype = 'pdat', mindate = inputTw_min, maxdate = inputTw_max)
    record_srch = Entrez.read(handle_srch)

    count = int(record_srch["Count"])
    print("Found %i results" % count)

    ##############Using searched id information to get A file with all information needed###################
    Papers_name = "_".join(["Papers", inputKw,inputTw_min.replace("/", "-"), "to", inputTw_max.replace("/","-"), ".txt"])
    batch_size = 10
    out_handle = open(Papers_name, "w")
    for start in range(0, count, batch_size):
        end = min(count, start + batch_size)
        print("Going to download record %i to %i" % (start + 1, end))
        fetch_handle = Entrez.efetch(
            db="pubmed",
            rettype="medline",
            retmode="text",
            retstart=start,
            retmax=batch_size,
            webenv=record_srch["WebEnv"],
            query_key=record_srch["QueryKey"],
        )
        data = fetch_handle.read()
        fetch_handle.close()
        out_handle.write(data)
    out_handle.close()
    return(Papers_name)



#############Read information of saved file#########################
#Input: Generated file with all information for papers
#Output: A csv file generated with all extracted infromation: title, author, publish date, and abstract
def Get_info_all(Papers_name): 
    All_info_op = open(Papers_name)
    All_info = All_info_op.read()
    All_info_op.close() 

    Aifs = All_info.split("\n")
    # Get the position of each record in the text
    RcID = [i for i,value in enumerate(list(map(len, Aifs))) if value == 0]
    print("The number of papers extracted is: %i " % (len(RcID) - 1))

    Info_df = pd.DataFrame(np.zeros(((len(RcID) - 1),4)), columns=['Title', 'Author', 'Publish_date','Abstract'])
    for paperi in range(len(RcID) - 1):
        #Set a id for interested paper
        #paperi = 4
        #Paper content
        print("Paper %i is running" % paperi)
        ppcon = Aifs[RcID[paperi]:RcID[paperi + 1]]
        Info_df.loc[paperi,:] = Get_info_each(ppcon)
    
    Info_name = "_".join(["Info_from",Papers_name]).replace("_.txt", ".csv")
    Info_df.to_csv(Info_name, index = False)

Manue_search()

# Part 2: Database and Query Module
Design and implement a database module that can import the CSV file to SQLite to build a database automatically. Then implement SQL code to query the publications by author’s name (i.e., input an author’s name and find out and return all his/her publications). 

In [ ]:
import csv
import sqlite3
from pathlib import Path

After importing the above modules, create an empty sqlite database

In [ ]:
Path('part2.db').touch()

Then, create a database connection and cursor to execute queries

In [ ]:
conn = sqlite3.connect('part2.db')
c = conn.cursor()

Next, execute a query that will create an "HIV_Papers" table with "Title", "Author", "Publish_date", and "Abstract" columns

In [ ]:
c.execute('''CREATE TABLE HIV_Papers (Title text, Author text, Publish_date date, Abstract text)''')

Now, load the CSV file into a dataframe. Then, write the data from the dataframe to the SQLite table.

In [ ]:
hivpapers = pd.read_csv('Info_from_Papers_HIV_2020-08-29_to_2020-08-30.csv')
hivpapers.to_sql('HIV_Papers', conn, if_exists='append', index = False)

Prompt user to input author name. If the author name is found, the output returns the author's name, all co-authors, and the title of the publication. If there are multiple results of that name, each result will show on a new line. If the author's name is not found, the output will give an error message and repeat the prompt input.

In [ ]:
while True:
    input_author = input('Enter an author name, e.g. Nguyen: ') 
    result = c.execute(f"SELECT Author, Title FROM HIV_Papers WHERE Author LIKE '%{input_author}%'")
    res = result.fetchall()
    if len(res)!=0:
        for row in res:
            print(row)
        break
    else:
        print("Sorry, the author name you entered does not match any authors.")

# Part 3: Visualization Dashboard 

#### First, let's import all the modules we might be requiring

In [ ]:
import pandas
import numpy
import scipy
import matplotlib
import matplotlib.pyplot as plt
from ipyfilechooser import FileChooser
import seaborn

#### Now, let's read the csv file we want to analyze today. Please choose the csv file you would like to analyze.

In [ ]:
# Create and display a FileChooser widget
fc = FileChooser()
display(fc)

In [ ]:
# read the csv file
CSV = pandas.read_csv(fc.selected)

# here are the first 3 rows of the file you chose to read
CSV.head(3)

#### Number of publications in each year

In [ ]:
# First split the string of publications
split_dates = CSV['Publish_date'].str.split('\ ')

# save how many publication we have
num_publications = len(split_dates)

# loop through all publications and extract dates
year_list = []
for i in range(num_publications):
    # get current publication year
    year_list.append( split_dates[i][0])


# get the vector of unique years
unique_years = list(set(year_list))

# make a data frame of counts per year
publications_per_year = pandas.DataFrame(columns = ["Year",
                                                    "Publications"])
for i in range(len(unique_years)):
    # get current year
    year = unique_years[i]
    
    # get current counts
    cnts = year_list.count(str(year))
    
    # make df
    pubs_this_year = pandas.DataFrame(data = [[year,
                                               cnts]], 
                                    columns = ["Year",
                                               "Publications"])
    
    # append to out data frame
    publications_per_year = publications_per_year.append(pubs_this_year,
                                                         ignore_index=True)

# show some rows
publications_per_year.head()

#### Trend of the publication numbers over time

In [ ]:
# Show publications over time
seaborn.histplot(year_list)
plt.xlabel("Year of Publication")
plt.ylabel("Number of Publications")

#### Generate and visualize the summary statistics for the publication number per year, including mean, SD, range, median, 1st to 3rd quartile.

In [ ]:
# Calculate Statistics
mn = publications_per_year.Publications.mean()
med = publications_per_year.Publications.median()
sdev = publications_per_year.Publications.std()
q25 = publications_per_year.Publications.quantile(0.25)
q75 = publications_per_year.Publications.quantile(0.75)

# Make a table
Statistics_Table = pandas.DataFrame(data = [[mn,
                                             med,
                                            sdev,
                                            q25,
                                            q75]],
                                    columns = ["Mean",
                                               "Median",
                                              'Standard Deviation',
                                              'First Quantile',
                                              'Third Quantile'])

# show table
Statistics_Table